# 1DoF Hamilton Saddle-Node

We study the _Hamilton Saddle-node_ (HSN) system with Hamiltonian

\begin{equation}
H = \frac{1}{2} p^2 - \sqrt{\mu}q^2 + \frac{1}{3}q^3
\end{equation}

and associated _Hamilton equations_

\begin{align}
\dot{q} &= p\\
\dot{p} &= 2\sqrt{\mu}q - q^2
\end{align}

The _equilibrium points_ for this system are located at $(0,0)$ (_saddle point_) and $(2\sqrt{\mu},0)$ (_centre point_), where $\mu$ is the _biurcation parameter_, which clearly controls the distance between these points. When, $\mu < 0$, no equilibrium points exist. 

## Phase Portraits

We're interested in the case when equilibrium points exist, since invariant manifolds are observed.

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy
from numpy import arange
from numpy import meshgrid

In [16]:
# Mesh of initial conditions in 2D plane
delta = 0.001
xrange = arange(-1, 2.0, delta)
yrange = arange(-1.5, 1.5, delta)
q0, p0 = meshgrid(xrange,yrange)

# Plot vector field
fig,ax = plt.subplots(2,1,figsize=(5,10),dpi=100)

#########################################
# Parameters for Ham Saddle-Node
MU = 0
ALPHA = 1
# Hamiltonian
H0 = 0.5*p0**2 - numpy.sqrt(MU)*q0**2 + ALPHA*q0**3/3
ax[0].set_title('HSN: $\mu = 0$', fontsize=25)
ax[0].set_xlabel('q', fontsize=20)
ax[0].set_ylabel('p', fontsize=20)
ax[0].contour(q0,p0,H0,40,cmap='jet')
#########################################
# Parameters for Ham Saddle-Node
MU = 0.25
ALPHA = 1
# Hamiltonian
H0 = 0.5*p0**2 - numpy.sqrt(MU)*q0**2 + ALPHA*q0**3/3
ax[1].set_title('HSN: $\mu = 0.25$', fontsize=25)
ax[1].set_xlabel('q', fontsize=20)
ax[1].set_ylabel('p', fontsize=20)
ax[1].contour(q0,p0,H0,40,cmap='jet')
#########################################
# Parameters for Ham Saddle-Node
# MU = 0.25
# ALPHA = 1
# # Hamiltonian
# H0 = 0.5*p0**2 - numpy.sqrt(MU)*q0**2 + ALPHA*q0**3/3
# ax[2].set_title('Phase-Space Portrait Hamilton-SN', fontsize=20)
# ax[2].set_xlabel('q', fontsize=20)
# ax[2].set_ylabel('p', fontsize=20)
# ax[2].contour(q0,p0,H0,40,cmap='Blues')

fig.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

# Numerical solution and Invertibility of $D_0^1$ data matrices

* Initial conditions: $(q_0, p_0) = (q(t_0), p(t_0))$, were sampled from square grid of $N \times N$ points in phase-space
* We integrated ONLY for a single timestep, $\Delta t$, i.e.,  $(q(t), p(t))$, with $t = t_0 + \Delta t$
* We used _Explicit Runge-Kutta_ of order 5, implemented as part of the Python library `scipy.integrate`

<span style='color:red'>CHECK</span>
* Meaning and values of the relative and absolute tolerance variables defined in numeric integrator
* Do I need quadruple presicion?

__CONSTRUCT__

_TASK 1_ (<span style='color:blue'>DONE</span>): Data matrices $D_0^1$ as before

\begin{equation*}
D_0^1 = 
\begin{bmatrix}
\begin{pmatrix}
q(t_0)\\
p(t_0)
\end{pmatrix}
\begin{pmatrix}
q(t_0 + \Delta t)\\
p(t_0 + \Delta t)
\end{pmatrix}
\end{bmatrix}
\end{equation*}

and compute their determinant $det(D_0^1)$ to tests for invertibility

_TASK 2_ : Compare the trajectory-based operator with the nonlinear flow

* Compute $\mathbb{E}[\hat{A}]$, with $\hat{A}$ defined according to DMD as before

\begin{equation*}
\hat{A} = 
\begin{pmatrix}
q(t_0 + \Delta t)\\
p(t_0 + \Delta t)
\end{pmatrix}
\begin{pmatrix}
q(t_0)\\
p(t_0)
\end{pmatrix}^+
\end{equation*}

Take average for all initial conditons

* Then, compute the output of this operator and compare with values from numerical solution 

\begin{equation*}
\mathbb{E}[\hat{A}] 
\begin{pmatrix}
q(t_0)\\
p(t_0)
\end{pmatrix}
\end{equation*}

Take the Euclidean distance to check the error in the output of using $\mathbb{E}[\hat{A}]$, that is, compute

\begin{equation*}
\lVert
\begin{pmatrix}
q(t_0 + \Delta t)\\
p(t_0 + \Delta t)
\end{pmatrix}
-
\mathbb{E}[\hat{A}]
\begin{pmatrix}
q(t_0)\\
p(t_0)
\end{pmatrix}
\rVert
\end{equation*}

## Case $\mu = 0$

### Routine for Numerical Soltution

In [397]:
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Parameters
MU = 0
ALPHA = 1
# Bundle parameters for ODE solver
params = [MU, ALPHA]

def f(y, t, params):
    q, p = y      # unpack current values of y
    MU, ALPHA = params  # unpack parameters
    derivs = [p, 2*numpy.sqrt(MU)*q - q**2]     # list of dy/dt=f functions
    return derivs
###########################################
#
# Meshgrid of initial conditions (q0,p0)
#
###########################################
# Grid limits
x_min, x_max = (-1.5, 1.5)
y_min, y_max = (-1.5, 1.5)
# Number of grid points per axis
N = 200
# Grid resolution per axis
x_res = (x_max - x_min)/float(N)
y_res = x_res
# Construct grid for the above params
x = np.arange(x_min, x_max, x_res)
y = np.arange(y_min, y_max, y_res)
X,Y = np.meshgrid(x, y)
###########################################
#
# Make time array for solution
#
###########################################
n_steps = 1
t0 = 0.
t_final = t0 + n_steps*dt
dt = 0.02 # Time increment
t = np.arange(t0, t_final+dt, dt)
###########################################
#
# Solve ODEs for every initial condition in grid
#
###########################################
D01_det_data = []
A_hat_data = []
grid_points = []
# trajectory = {}
for i in range(len(X[0])):
    trajectory[i] = {}    
    for j in range(len(Y[0])):
        q0 = float(X[i][j])
        p0 = float(Y[i][j])
        grid_points.append([q0, p0])
        ###########################################
        #
        # Solve ODEs for initial conditions (q0, p0)
        #
        ###########################################
        # Bundle initial conditions for ODE solver
        y0 = [q0, p0]
        # Call ODE solver
        solution = odeint(f, y0, t, args=(params,))
        # Vector for next step in evolution and its P-inverse
#         trajectory[i][j] = solution
        ###########################################
        #
        # Data matrices and their determinant 
        #
        ###########################################
        # Column vector of IC.s and its MP-pseudoinverse
        q0, p0 = solution[0]
        r0 = numpy.matrix([q0, p0]).T
        q, p = solution[1]
        r = numpy.matrix([q, p]).T
        # Data matrix D_0^1
        D01 = numpy.asmatrix(numpy.concatenate([[r0],[r]])).T
        D01_det = numpy.linalg.det(D01)
        D01_det_data.append(D01_det)
        ###########################################
        #
        # Trajectory-based operator for IC
        #
        ###########################################
        r0_pinv = r0.T/float(numpy.linalg.norm(r0)**2)
        A_hat = numpy.matmul(r, r0_pinv)

### Invertibility Plot of $D_0^1$ matrices

In [398]:
import matplotlib
fig,ax = plt.subplots(2,2,figsize=(10,8), sharex =True, sharey=True)

x,y = numpy.asarray(grid_points).T

############################################
i,j=(0,0)
ax[i][j].scatter(x,y,c=D01_det_data,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title('det($D_0^1$), $\mu$=0', fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)

############################################
p1 = 0.7
i,j=(0,1)
ax[i][j].scatter(x,y,c=numpy.abs(D01_det_data)**p1,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title("$det(D_0^1)^p$, "+"p="+str(p1)+", $\mu$=0", fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)

############################################
p2 = 0.5
i,j=(1,0)
ax[i][j].scatter(x,y,c=numpy.abs(D01_det_data)**p2,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title("$det(D_0^1)^p$, "+"p="+str(p2)+", $\mu$=0", fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)


############################################
p3 = 0.1
i,j=(1,1)
ax[i][j].scatter(x,y,c=numpy.abs(D01_det_data)**p3,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title("$det(D_0^1)^p$, "+"p="+str(p3)+", $\mu$=0", fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)


fig.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

### Dynamics generated by $\mathbb{E}[\hat{A}]$ VS $\phi_{\Delta t}$

<span style='color:red'>UNDER CONSTRUCTION</span>

## Case $\mu > 0$

### Routine for Numerical Soltution

In [410]:
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Parameters
MU = 0.25
ALPHA = 1
# Bundle parameters for ODE solver
params = [MU, ALPHA]

def f(y, t, params):
    q, p = y      # unpack current values of y
    MU, ALPHA = params  # unpack parameters
    derivs = [p, 2*numpy.sqrt(MU)*q - q**2]     # list of dy/dt=f functions
    return derivs
###########################################
#
# Meshgrid of initial conditions (q0,p0)
#
###########################################
# Grid limits
x_min, x_max = (-1.5, 1.5)
y_min, y_max = (-1.5, 1.5)
# Number of grid points per axis
N = 200
# Grid resolution per axis
x_res = (x_max - x_min)/float(N)
y_res = x_res
# Construct grid for the above params
x = np.arange(x_min, x_max, x_res)
y = np.arange(y_min, y_max, y_res)
X,Y = np.meshgrid(x, y)
###########################################
#
# Make time array for solution
#
###########################################
n_steps = 1
t0 = 0.
t_final = t0 + n_steps*dt
dt = 0.02 # Time increment
t = np.arange(t0, t_final+dt, dt)
###########################################
#
# Solve ODEs for every initial condition in grid
#
###########################################
D01_det_data = []
A_hat_data = []
grid_points = []
# trajectory = {}
for i in range(len(X[0])):
    trajectory[i] = {}    
    for j in range(len(Y[0])):
        q0 = float(X[i][j])
        p0 = float(Y[i][j])
        grid_points.append([q0, p0])
        ###########################################
        #
        # Solve ODEs for initial conditions (q0, p0)
        #
        ###########################################
        # Bundle initial conditions for ODE solver
        y0 = [q0, p0]
        # Call ODE solver
        solution = odeint(f, y0, t, args=(params,))
        # Vector for next step in evolution and its P-inverse
#         trajectory[i][j] = solution
        ###########################################
        #
        # Data matrices and their determinant 
        #
        ###########################################
        # Column vector of IC.s and its MP-pseudoinverse
        q0, p0 = solution[0]
        r0 = numpy.matrix([q0, p0]).T
        q, p = solution[1]
        r = numpy.matrix([q, p]).T
        # Data matrix D_0^1
        D01 = numpy.asmatrix(numpy.concatenate([[r0],[r]])).T
        D01_det = numpy.linalg.det(D01)
        D01_det_data.append(D01_det)
        ###########################################
        #
        # Trajectory-based operator for IC
        #
        ###########################################
        r0_pinv = r0.T/float(numpy.linalg.norm(r0)**2)
        A_hat = numpy.matmul(r, r0_pinv)

### Invertibility Plot of $D_0^1$ matrices

In [411]:
import matplotlib
fig,ax = plt.subplots(2,2,figsize=(10,8), sharex =True, sharey=True)

x,y = numpy.asarray(grid_points).T

############################################
i,j=(0,0)
ax[i][j].scatter(x,y,c=D01_det_data,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title('det($D_0^1$), $\mu$=0.25', fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)

############################################
p1 = 0.7
i,j=(0,1)
ax[i][j].scatter(x,y,c=numpy.abs(D01_det_data)**p1,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title("$det(D_0^1)^p$, "+"p="+str(p1)+", $\mu$=0.25", fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)

############################################
p2 = 0.5
i,j=(1,0)
ax[i][j].scatter(x,y,c=numpy.abs(D01_det_data)**p2,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title("$det(D_0^1)^p$, "+"p="+str(p2)+", $\mu$=0.25", fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)


############################################
p3 = 0.1
i,j=(1,1)
ax[i][j].scatter(x,y,c=numpy.abs(D01_det_data)**p3,s=15,cmap='bone')

ax[i][j].plot([0,0],[-1.5,1.5],'--')
ax[i][j].plot([2*numpy.sqrt(MU), 2*numpy.sqrt(MU)],[-1.5,1.5],'--')

ax[i][j].set_title("$det(D_0^1)^p$, "+"p="+str(p3)+", $\mu$=0.25", fontsize=20)
ax[i][j].set_xlabel('$q_0$',fontsize=20)
ax[i][j].set_ylabel('$p_0$',fontsize=20)

fig.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

### Dynamics generated by $\mathbb{E}[\hat{A}]$ VS $\phi_{\Delta t}$

<span style='color:red'>UNDER CONSTRUCTION</span>

# SCRATCH NOTES

## Notes: Group Meeting

* Use RK 4order
* Check energy conservation

In [381]:
Z0 = np.zeros_like(X)
Z1 = np.zeros_like(X)
dZ = np.zeros_like(X)
for i in range(len(X[0])):
    for j in range(len(Y[0])):
        r0,r1 = trajectory[i][j]
        q0,p0 = r0
        H0 = 0.5*p0**2 - numpy.sqrt(MU)*q0**2 + ALPHA*q0**3/3
        q1,p1 = r1
        H1 = 0.5*p1**2 - numpy.sqrt(MU)*q1**2 + ALPHA*q1**3/3
#         speed = numpy.linalg.norm(q1 - q0)
        Z0[i][j] = H0
        Z1[i][j] = H1
        dZ[i][j] = abs(H1-H0)

ValueError: setting an array element with a sequence.

## Runge-Kutta Implementation

In [177]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import RK45

# Parameters
MU = 0.0
ALPHA = 1

def f(t, y):
    q, p = y      # unpack current values of y
    MU, ALPHA = params  # unpack parameters
    derivs = [p, 2*numpy.sqrt(MU)*q - q**2] # list of dy/dt=f functions
    return derivs

# Meshgrid of initial conditions (q0,p0)
x_min, x_max = (-1.5, 1.5)
y_min, y_max = (-1.5, 1.5)

N = 100
x_res = (x_max - x_min)/float(N)
y_res = x_res

x = np.arange(x_min, x_max, x_res)
y = np.arange(y_min, y_max, y_res)
X,Y = np.meshgrid(x, y, sparse=True)

D01_det_data = []
grid_points = []
data = []
for i in range(len(X[0])):
    q0 = float(X[0][i])
    for j in range(len(Y)):
        p0 = float(Y[j])
        grid_points.append([q0, p0])
        # Column vector of IC.s and its MP-pseudoinverse
        r0 = numpy.matrix([q0,p0]).T
        r0_pinv = r0.T/float(numpy.linalg.norm(r0))
        
        # Bundle parameters for ODE solver
        params = [MU, ALPHA]

        # Bundle initial conditions for ODE solver
        y0 = [q0, p0]

        # Make time array for solution
        dt = 0.02 # Increment
        t_initial = 0
        t_final = dt

        # Call the ODE solver
        solution = RK45(f,t_initial,y0,t_final)
        # collect data
        t_values = []
        y_values = []
        for i in range(100):
            # get solution step state
            solution.step()
            t_values.append(solution.t)
            y_values.append(solution.y)
            # break loop after modeling is finished
            if solution.status == 'finished':
                break

        data.append([t_values, y_values[-1]])
        
        # Vector for next step in evolution and its P-inverse
#         q, p = psoln[1]
#         r = numpy.matrix([q, p]).T
        
#         # Trajectory-based operator for IC
#         A_hat = numpy.matmul(r, r0_pinv)
        
#         # Data matrix D_0^1
#         D01 = numpy.asmatrix(numpy.concatenate([[r0],[r]])).T
#         D01_det = numpy.linalg.det(D01)
#         D01_det_data.append(D01_det)
        
#         ax.scatter(q0,p0,c='blue',s=5)
#         twopi = 1.7
#         ax.plot(psoln[:,0], psoln[:,1],c='green')
#     #     ax.scatter(psoln[1][0], psoln[1][1],'o',c='blue')

# # ax.plot([-numpy.sqrt(MU), -numpy.sqrt(MU)],[-0.6,0.6],'--',c='blue',label='q: Saddle point')
# ax.set_xlabel('q', fontsize=20)
# ax.set_ylabel('p', fontsize=20)
# ax.set_xlim(-1.55, twopi)
# # ax.legend()

# plt.tight_layout()
# plt.show()

In [178]:
D01_det_data = []
for i in range(len(X[0])):
    q0 = float(X[0][i])
    for j in range(len(Y)):
        p0 = float(Y[j])
        # Column vector of IC.s and its MP-pseudoinverse
        r0 = numpy.matrix([q0, p0]).T
        r0_pinv = r0.T/float(numpy.linalg.norm(r0))
        
        # Vector for next step in evolution and its P-inverse
        q, p = data[i][-1]
        r = numpy.matrix([q, p]).T
        
        # Trajectory-based operator for IC
        A_hat = numpy.matmul(r, r0_pinv)
        
        # Data matrix D_0^1
        D01 = numpy.asmatrix(numpy.concatenate([[r0],[r]])).T
        D01_det = numpy.linalg.det(D01)
        D01_det_data.append(D01_det)

## Random Sampling of Initial Conditions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def f(y, t, params):
    q, p = y      # unpack current values of y
    MU, ALPHA = params  # unpack parameters
    derivs = [p, MU - q**2]     # list of dy/dt=f functions
    return derivs

# Parameters
MU = 0.25
ALPHA = 1

# Initial values
# Parameters for non-closed path
#q0 = -numpy.sqrt(MU)+0.001 # initial angular displacement
#p0 = 0.01    # initial angular velocity

# Parameters closed-path
# q0 = -numpy.sqrt(MU)-0.05 # initial angular displacement
# p0 = 0.0    # initial angular velocity

# Plot results
fig,ax = plt.subplots(1,1,figsize=(8,8))

R0 = 2.5*numpy.random.rand(2,10000)-1 # Initial conditions
R0 = R0.T

D01_det_data = []
for i in range(len(R0)):
    q0, p0 = R0[i]
    
    # Column vector of IC.s and its MP-pseudoinverse
    r0 = numpy.matrix([q0,p0]).T
    r0_pinv = r0.T/float(numpy.linalg.norm(r0))
    
    # Bundle parameters for ODE solver
    params = [MU, ALPHA]

    # Bundle initial conditions for ODE solver
    y0 = [q0, p0]

    # Make time array for solution
    tStop = 0.04
    tInc = 0.02
    t = np.arange(0., tStop, tInc)

    # Call the ODE solver
    psoln = odeint(f, y0, t, args=(params,))
    
    # Vector for next step in evolution and its P-inverse
    q, p = psoln[1]
    r = numpy.matrix([q, p]).T
    
    # Trajectory-based operator for IC
    A_hat = numpy.matmul(r, r0_pinv)
    
    # Data matrix D_0^1
    D01 = numpy.asmatrix(numpy.concatenate([[r0],[r]])).T
    D01_det = numpy.linalg.det(D01)
    D01_det_data.append(D01_det)
    
    ax.scatter(q0,p0,c='blue')
    twopi = 1.7
    ax.plot(psoln[:,0], psoln[:,1],c='green')
#     ax.scatter(psoln[1][0], psoln[1][1],'o',c='blue')
    

# ax.plot([-numpy.sqrt(MU), -numpy.sqrt(MU)],[-0.6,0.6],'--',c='blue',label='q: Saddle point')
ax.set_xlabel('q', fontsize=20)
ax.set_ylabel('p', fontsize=20)
ax.set_xlim(-1.5, twopi)
# ax.legend()

plt.tight_layout()
plt.show()

## Code block to plot $det(D_0^1)$

In [237]:
import matplotlib
fig,ax = plt.subplots(1,1,figsize=(10,8))

x,y = numpy.asarray(grid_points).T
s = ax.scatter(x,y,c = numpy.abs(D01_det_data)**0.1,s=15,cmap='bone')
ax.plot([-numpy.sqrt(MU), -numpy.sqrt(MU)],[-1.5,1.5],'--')
ax.plot([numpy.sqrt(MU), numpy.sqrt(MU)],[-1.5,1.5],'--')
ax.set_title('det($D_0^1$)', fontsize=20)
ax.set_xlabel('$q_0$',fontsize=20)
ax.set_ylabel('$p_0$',fontsize=20)
fig.colorbar(s)
plt.show()

<IPython.core.display.Javascript object>